In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pathlib import Path

pd.set_option("display.max_columns", 500)

In [2]:
df = pd.read_csv(Path("data", "combo.csv"))

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns

Index(['npi', 'lastname', 'firstname', 'credentials', 'gender', 'address',
       'city', 'zip', 'state', 'provider_type', 'place_of_service',
       'hcpcs_code', 'hcpcs_description', 'line_srvc_cnt', 'bene_unique_cnt',
       'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt', 'Total_submitted_chrg_amt',
       'Total_Medicare_payment_amt', 'EXCLTYPE', 'year', 'Target'],
      dtype='object')

In [4]:
df.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year,Target
0,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502.0,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,None,None,0
1,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502.0,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,None,None,0
2,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502.0,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,None,None,0
3,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502.0,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,None,None,0
4,1003000142,khalil,rashid,M.D.,M,4126 n holland sylvania rd,toledo,43623.0,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,None,None,0


In [ ]:
df.pivot_table(index='npi', columns='hcpcs_code', 
         values=['line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt', \
                 'average_Medicare_allowed_amt', 'average_submitted_chrg_amt', \
                 'average_Medicare_payment_amt', 'average_Medicare_standard_amt',\
                 'Total_submitted_chrg_amt', 'Total_Medicare_payment_amt'])

In [ ]:
df1

In [ ]:
df.groupby('hcpcs_code')['line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
'average_submitted_chrg_amt', 'average_Medicare_payment_amt', 'average_Medicare_standard_amt','Total_submitted_chrg_amt', 'Total_Medicare_payment_amt']
